In [7]:
import numpy as np
import h5py
import os, sys
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.signal import savgol_filter
import pandas as pd
sys.path.append('/Users/ajc/codes/simulator/debug/ats/tools/utils/')
#sys.path.append('/Users/ajc/CODES/ats/tools/utils/')
import get_transect_data
path='/Users/ajc/NFS/simulations/hillslope/2019/MaxSnow/'

In [6]:
Data_B = dict()
#infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_B_1R1/'
infile=os.path.join(path,'MS_B_1R1')
Time, Data = get_transect_data.get2DMatrix(infile)
Data_B['temperature'] = np.array(Data)
Data_B['time'] = np.array(Time)

<HDF5 file "visdump_surface_data.h5" (mode r)>
Mesh type:  HEX
3 7037 100 81


In [8]:
Data_S30 = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S30_1R1/'
infile=os.path.join(path,'MS_S30_1R1')
Time, Data = get_transect_data.get2DMatrix(infile)
Data_S30['temperature'] = np.array(Data)
Data_S30['time'] = np.array(Time)

<HDF5 file "visdump_surface_data.h5" (mode r)>
Mesh type:  HEX
3 7037 100 81


In [9]:
Data_S60 = dict()
#infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S60_1R1/'
infile=os.path.join(path,'MS_S60_1R1')
Time, Data = get_transect_data.get2DMatrix(infile)
Data_S60['temperature'] = np.array(Data)
Data_S60['time'] = np.array(Time)

<HDF5 file "visdump_surface_data.h5" (mode r)>
Mesh type:  HEX


RuntimeError: Can't decrement id ref count (unable to close file, errno = 5, error message = 'Input/output error')

In [ ]:
Data_S90 = dict()
#infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S90_1R1/'
infile=os.path.join(path,'MS_S60_1R1')
Time, Data = get_transect_data.get2DMatrix(infile)
Data_S90['temperature'] = np.array(Data)
Data_S90['time'] = np.array(Time)

In [ ]:
def get_AveMaxMin_Temp(season = " ", start=0, end=-1,depth=25):
    #25 * 2cm = 50 cm
    data = []
    times = []
    if season == "B":
        data = Data_B['temperature']
        times = Data_B['time']
    elif season == "S30":
        data = Data_S30['temperature']
        times = Data_S30['time']
    elif season == "S60":
        data = Data_S60['temperature']
        times = Data_S60['time']
    elif season == "S90":
        data = Data_S90['temperature']
        times = Data_S90['time']
        
    nvar, cycles, xnum, znum = data.shape
    HillslopeT = dict()
    
    AvgTemp = np.zeros(cycles)
    MaxTemp = np.zeros(cycles)
    MinTemp = np.zeros(cycles)
    #print depth
    depth_cm = data[1,0,0,80] - data[1,0,0,80-depth]
    for i in range(cycles):
        AvgTemp[i] = np.mean(data[nvar-1,i,start:end,80-depth:])
        MaxTemp[i] = np.max(data[nvar-1,i,start:end,80-depth:])
        MinTemp[i] = np.min(data[nvar-1,i,start:end,80-depth:])
    L = len(AvgTemp)%365
    HillslopeT['times'] = np.array(times[:-L])
    HillslopeT_Avg = np.array(AvgTemp[:-L])
    HillslopeT_Max = np.array(MaxTemp[:-L])
    HillslopeT_Min = np.array(MinTemp[:-L])
   

    HillslopeT['temp-annual_avg'] = np.reshape(HillslopeT_Avg, (-1,365)).mean(axis=1)
    HillslopeT['temp-annual_max'] = np.reshape(HillslopeT_Max, (-1,365)).max(axis=1)
    HillslopeT['temp-annual_min'] = np.reshape(HillslopeT_Min, (-1,365)).min(axis=1)

    return HillslopeT, depth_cm

In [ ]:
def get_Temp_timeseries_location(season = " ", location=0,depth=25):
    #25 * 2cm = 50 cm
    data = []
    times = []
    if season == "B":
        data = Data_B['temperature']
        times = Data_B['time']
    elif season == "S30":
        data = Data_S30['temperature']
        times = Data_S30['time']
    elif season == "S60":
        data = Data_S60['temperature']
        times = Data_S60['time']
    elif season == "S90":
        data = Data_S90['temperature']
        times = Data_S90['time']
        
    nvar, cycles, xnum, znum = data.shape
    HillslopeT1 = dict()
    
    Temp = np.zeros(cycles)
    
    #print depth
    depth_cm = data[1,0,0,80] - data[1,0,0,80-depth]
    for i in range(cycles):
        Temp[i] = data[nvar-1,i,location,80-depth]

    HillslopeT1['times'] = np.array(times[-365:])
    HillslopeT_Avg = np.array(Temp[-365:])

    HillslopeT1['temp'] = HillslopeT_Avg
    
    return HillslopeT1, depth_cm

In [ ]:
def get_locationbased_ALT(season = " ",location=0):
    data = []
    times = []
    if season == "B":
        data = Data_B['temperature']
        times = Data_B['time']
    elif season == "S30":
        data = Data_S30['temperature']
        times = Data_S30['time']
    elif season == "S60":
        data = Data_S60['temperature']
        times = Data_S60['time']
    elif season == "S90":
        data = Data_S90['temperature']
        times = Data_S90['time']
        
    nvar, cycles, xnum, znum = data.shape
    HillslopeT = dict()
    avgTemp = np.zeros(cycles)
    
    ddepth = 0 
    for i in range(cycles):
        loc = np.where(data[nvar-1,i,location,:] > 273.15)[0]
        if len(loc) >0:
            dz = -(80 - loc[0])*2.0
            avgTemp[i] = dz
    HillslopeT['times'] = np.array(times)
    HillslopeT['temp'] = np.array(avgTemp)
    print nvar,len(loc), loc 
    print data[1,0,location,:], data[1,0,location,loc[0]], loc[0]
    L = len(HillslopeT['temp'])%365
    for key in HillslopeT.keys():
        HillslopeT[key + '-annual_avg'] = np.reshape(HillslopeT[key][:-L], (-1,365))   
    return HillslopeT

In [ ]:
start1_,end1_,depth_ = 0,5,55 # Z depth 3.84718
#start1_,end1_,depth_ = 0,5,46 # Z depth 2.053

WL_AMM_B, Depth_cm = get_AveMaxMin_Temp("B",start=start1_,end=end1_,depth=depth_)
WL_AMM_S30,Depth_cm = get_AveMaxMin_Temp("S30",start=start1_,end=end1_,depth=depth_)
WL_AMM_S60,Depth_cm = get_AveMaxMin_Temp("S60",start=start1_,end=end1_,depth=depth_)
WL_AMM_S90,Depth_cm = get_AveMaxMin_Temp("S90",start=start1_,end=end1_,depth=depth_)
print (Depth_cm)

In [ ]:
depth_cm = round(100*Depth_cm)

In [ ]:
UplandTemp = dict()
start2_,end2_ = end1_,99
UL_AMM_B,Depth_cm = get_AveMaxMin_Temp("B",start=start2_,end=end2_,depth=depth_)
UL_AMM_S30,Depth_cm = get_AveMaxMin_Temp("S30",start=start2_,end=end2_,depth=depth_)
UL_AMM_S60,Depth_cm = get_AveMaxMin_Temp("S60",start=start2_,end=end2_,depth=depth_)
UL_AMM_S90,Depth_cm = get_AveMaxMin_Temp("S90",start=start2_,end=end2_,depth=depth_)

In [ ]:
depth1=26
WL_T_B, Depth_cm1 = get_Temp_timeseries_location("B",location=0,depth=depth1)
WL_T_S30,Depth_cm1 = get_Temp_timeseries_location("S30",location=0,depth=depth1)
WL_T_S60,Depth_cm1 = get_Temp_timeseries_location("S60",location=0,depth=depth1)
WL_T_S90,Depth_cm1 = get_Temp_timeseries_location("S90",location=0,depth=depth1)
print Depth_cm1

In [ ]:
depth1=26
UL_T_B, Depth_cm1 = get_Temp_timeseries_location("B",location=49,depth=depth1)
UL_T_S30,Depth_cm1 = get_Temp_timeseries_location("S30",location=49,depth=depth1)
UL_T_S60,Depth_cm1 = get_Temp_timeseries_location("S60",location=49,depth=depth1)
UL_T_S90,Depth_cm1 = get_Temp_timeseries_location("S90",location=49,depth=depth1)
print Depth_cm1

In [ ]:
# Plot average temperature, minimum temperature, and time series: one column for wetlands only
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
def Plot_Wetlands():
    fig, axs = plt.subplots(3,1, figsize=(6,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    TempC = 273.15
    plt.tight_layout()
    axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Xticks = ['Basecase', r'S$_{30}$', r'S$_{60}$', r'S$_{90}$']
    
    ax1=plt.subplot(311)
    X = [RiparianTempB['temp-annual_avg'][-1], RiparianTempS30['temp-annual_avg'][-1],RiparianTempS60['temp-annual_avg'][-1],RiparianTempS90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['k', 'g','b','r'], width=0.25,edgecolor='black')
    
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax1.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=12, fontweight='normal')
    #ax1.set_ylim((266.5 - TempC, 271.5- TempC))
    ax1.set_ylim((-3.0, 0))
    #ax1.set_title('Wetland',fontweight='normal')
    ax1.text(0.5, -2.85, 'Annual average (wetland)',fontsize=11, fontweight='normal')
    plt.tight_layout()
    
    # --------------------------------- MANIMUM ---------------------------
    ax3=plt.subplot(312)
    X = [RiparianTempB['temp-annual_min'][-1], RiparianTempS30['temp-annual_min'][-1],RiparianTempS60['temp-annual_min'][-1],RiparianTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax3.bar(xpos,X,align='center', alpha=1.0, color=['k', 'g','b','r'], width=0.25,edgecolor='black')
       
    ax3.set_xticks(xpos)
    ax3.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax3.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=12, fontweight='normal')
    
    ax3.set_ylim((-15, -10))
    ax3.text(0.5, -14.85, 'Annual minimum (wetland)',fontsize=11, fontweight='normal')
    
    # --------------------------------- Annual time series ---------------------------
    ax3=plt.subplot(313)
    x=np.linspace(0,364,365)
    ax3.plot(x,-273.15+WetlandTempB['temp'],'k-',label='Basecase')
    ax3.plot(x,-273.15+WetlandTempS30['temp'],'g-',label=r'S$_{30}$')
    ax3.plot(x,-273.15+WetlandTempS60['temp'],'b-',label=r'S$_{60}$')
    ax3.plot(x,-273.15+WetlandTempS90['temp'],'r-',label=r'S$_{90}$')

    ax3.set_xlim(75,290)
    #ax3.set_xticks(np.linspace(1,19.6,6))
    #ax3.set_xticklabels(np.linspace(19.2*,365,6),fontsize=11, fontweight='normal')
    
    ax3.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=12, fontweight='normal')
    ax3.set_xlabel('Time [day of year]',fontsize=12, fontweight='normal')
    ax3.text(76., -4.9, 'Depth = 56 cm',fontsize=11, fontweight='normal')
    #ax3.set_ylim((-15, -10))
    ax3.legend(loc='lower right', fontsize=10,labelspacing=0.1,bbox_to_anchor=(0.9,-0.01, .1, .1))
    plt.tight_layout()
    
    
    plt.savefig(outfile+'avg-min-temp-385cm-bar.pdf', bbox_inches='tight', dpi=100)
    #riparian zone are more warmer due to sensible heat, support it that ALT (thawing) is more in the hillslope then if its not
    #warmer than either evaporative cooling or energy is lost in thawing, is it more thaw there then?

In [ ]:
# Plot average temperature, minimum temperature, and time series: columns for wetlands and uplands each
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
def Plot_Wetlands_Uplands():
    fig, axs = plt.subplots(3,2, figsize=(6,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    TempC = 273.15
    plt.tight_layout()
    axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Xticks = ['Basecase', r'S$_{30}$', r'S$_{60}$', r'S$_{90}$']
    
    ax1=plt.subplot(321)
    X = [WL_AMM_B['temp-annual_avg'][-1], WL_AMM_S30['temp-annual_avg'][-1],WL_AMM_S60['temp-annual_avg'][-1],WL_AMM_S90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['k', 'g','b','r'], width=0.25,edgecolor='black')
    
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax1.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=12, fontweight='normal')
    #ax1.set_ylim((266.5 - TempC, 271.5- TempC))
    ax1.set_ylim((-3, 0))
    ax1.set_title('Wetland',fontweight='normal')
    ax1.text(0.5, -2.85, 'Annual average',fontsize=10, fontweight='normal')
    plt.tight_layout()
    
    # Up land
    ax1=plt.subplot(322)
    X = [UL_AMM_B['temp-annual_avg'][-1], UL_AMM_S30['temp-annual_avg'][-1],UL_AMM_S60['temp-annual_avg'][-1],UL_AMM_S90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['k', 'g','b','r'], width=0.25,edgecolor='black')
    
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    #ax1.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=12, fontweight='normal')
    #ax1.set_ylim((266.5 - TempC, 271.5- TempC))
    ax1.set_ylim((-7, 0))
    ax1.set_title('Upland',fontweight='normal')
    ax1.text(0.5, -6.75, 'Annual average',fontsize=10, fontweight='normal')
    plt.tight_layout()
    
    # --------------------------------- MANIMUM ---------------------------
    ax3=plt.subplot(323)
    X = [WL_AMM_B['temp-annual_min'][-1], WL_AMM_S30['temp-annual_min'][-1],WL_AMM_S60['temp-annual_min'][-1],WL_AMM_S90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax3.bar(xpos,X,align='center', alpha=1.0, color=['k', 'g','b','r'], width=0.25,edgecolor='black')
       
    ax3.set_xticks(xpos)
    ax3.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax3.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=12, fontweight='normal')
    
    ax3.set_ylim((-15, -10))
    ax3.text(0.5, -14.85, 'Annual minimum',fontsize=10, fontweight='normal')
    
    #Upland
    ax3=plt.subplot(324)
    X = [UL_AMM_B['temp-annual_min'][-1], UL_AMM_S30['temp-annual_min'][-1],UL_AMM_S60['temp-annual_min'][-1],UL_AMM_S90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax3.bar(xpos,X,align='center', alpha=1.0, color=['k', 'g','b','r'], width=0.25,edgecolor='black')
       
    ax3.set_xticks(xpos)
    ax3.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    #ax3.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=12, fontweight='normal')
    
    ax3.set_ylim((-20, -10))
    ax3.text(0.5, -19.65, 'Annual minimum',fontsize=10, fontweight='normal')
    
    
    # --------------------------------- Annual time series ---------------------------
    ax3=plt.subplot(325)
    x=np.linspace(0,364,365)
    ax3.plot(x,-273.15+WL_T_B['temp'],'k-',label='Basecase')
    ax3.plot(x,-273.15+WL_T_S30['temp'],'g-',label=r'S$_{30}$')
    ax3.plot(x,-273.15+WL_T_S60['temp'],'b-',label=r'S$_{60}$')
    ax3.plot(x,-273.15+WL_T_S90['temp'],'r-',label=r'S$_{90}$')

    ax3.set_xlim(0,365)
    #ax3.set_xticks(np.linspace(1,19.6,6))
    #ax3.set_xticklabels(np.linspace(19.2*,365,6),fontsize=11, fontweight='normal')
    
    ax3.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=12, fontweight='normal')
    ax3.set_xlabel('Time [day of year]',fontsize=12, fontweight='normal')
    ax3.text(10, 0.5, 'Depth = 56 cm',fontsize=9, fontweight='normal')
    ax3.set_ylim((-5, 1))
    ax3.legend(loc='lower right', fontsize=8,labelspacing=0.01,bbox_to_anchor=(0.9,-0.01, .1, .1))
    plt.tight_layout()
    
    #upland
    ax3=plt.subplot(326)
    x=np.linspace(0,364,365)
    ax3.plot(x,-273.15+UL_T_B['temp'],'k-',label='Basecase')
    ax3.plot(x,-273.15+UL_T_S30['temp'],'g-',label=r'S$_{30}$')
    ax3.plot(x,-273.15+UL_T_S60['temp'],'b-',label=r'S$_{60}$')
    ax3.plot(x,-273.15+UL_T_S90['temp'],'r-',label=r'S$_{90}$')

    ax3.set_xlim(0,365)
    #ax3.set_xticks(np.linspace(1,19.6,6))
    #ax3.set_xticklabels(np.linspace(19.2*,365,6),fontsize=11, fontweight='normal')
    
    #ax3.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=12, fontweight='normal')
    ax3.set_xlabel('Time [day of year]',fontsize=12, fontweight='normal')
    ax3.text(10, -.5, 'Depth = 56 cm',fontsize=9, fontweight='normal')
    ax3.set_ylim((-17, 1))
    ax3.legend(loc='lower right', fontsize=8,labelspacing=0.01,bbox_to_anchor=(0.9,-0.01, .1, .1))
    plt.tight_layout()
    
    
    plt.savefig(outfile+'avg-min-temp-385cm-plot1.pdf', bbox_inches='tight', dpi=100)
    #riparian zone are more warmer due to sensible heat, support it that ALT (thawing) is more in the hillslope then if its not
    #warmer than either evaporative cooling or energy is lost in thawing, is it more thaw there then?

In [ ]:
Plot_Wetlands_Uplands()